In [44]:
# Standard Python libs
import pandas as pd
import numpy as np
from datetime import date, datetime
from time import mktime
import time
import pytz
# Machine Learning Libs
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import RMSprop, Adam, Nadam

#Trading libraries
import talib.abstract as ta
import poloniex
from tradingWithPython import backtest as bt
np.zeros((15,2))

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [42]:
symbol = 'BTC_ETH'
start = mktime(datetime(2015, 8,8).timetuple())
end = time.time()
pd.to_datetime(start*1e9)

Timestamp('2015-08-08 06:00:00')

In [3]:
polo = poloniex.Poloniex()

In [4]:
# joblib.dump(df_daily, '../data/BTCETHdf.pkl')

In [15]:
def load_data(symbols = ['USDT_ETH'], start = None, end= None, period= None, quantity= None, is_pkld = True):
    '''
    5 largest Cryptos by market volume = ['BTC', 'ETH', 'XRP', 'ETC', 'XMR']

    INSERT DOCSTRING HERE
    '''
    if is_pkld:
        prices = joblib.load('../data/prices_{}coins.pkl'.format(len(symbols)))
    else:
        sym_price = []
        a = []
        b = []
        for i in symbols:
            c = pd.DataFrame.from_records(Poloniex().returnChartData(currencyPair = 'USDT_' + i, start = start, end = end, period = period), index = 'date')
            c.index = pd.to_datetime(c.index*1e9)
            sym_price.append(c)
            a.extend([i]*len(c.columns))
            b.extend(list(c.columns))
        # enforce uniform length
        length = min([len(i) for i in sym_price])
        sym_price = [i[-length:] for i in sym_price]

        # Form MultiIndexed DataFrame
        prices = pd.DataFrame(data = np.hstack([i.values for i in sym_price]), columns = pd.MultiIndex.from_tuples([i for i in zip(a,b)]))
        joblib.dump(prices, '../data/prices_{}coins.pkl'.format(len(symbols)))
    X_train, X_test = self.split_data(prices, min(3000, len(prices)), 0.2)

    return X_train, X_test

In [ ]:
load_data(symbols = ['BTC','ETH'], start = start, end = end, period = 14400, quantity )

In [6]:
df_daily = joblib.load('../data/BTCETHdf.pkl')
quantity = 3000
X_train = df_daily.iloc[-quantity:-round(quantity*0.2)]
X_test  = df_daily.iloc[-quantity:,]

In [7]:
# Initializing state values
def init_state(X, test = False):
    close = X['close'].values
    diff = np.diff(close)
    diff = np.insert(diff, 0,0)
    sma15 = ta.SMA(X, timeperiod = 15)
    sma60 = ta.SMA(X, timeperiod = 60)
    rsi = ta.RSI(X, timeperiod = 14)
    atr = ta.ATR(X, timeperiod = 14)
    
    data = np.nan_to_num(np.vstack((close, diff, sma15, close-sma15, sma15-sma60, rsi, atr)))
    if not test:
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)
        data = np.expand_dims(scaled_data, axis = 1)
        joblib.dump(scaler, '../data/scaler.pkl')
    elif test:
        scaler = joblib.load('../data/scaler.pkl')
        scaled_data = scaler.transform(data)
        data = np.expand_dims(scaled_data, axis = 1)
    data = data.T
    print(data.shape)
    state = data[0:1,0:1,:]
    return state, data, close
init_state(X_train)[0]

(2400, 1, 7)


array([[[ 2.44948974, -0.40824829, -0.40824829, -0.40824829, -0.40824829,
         -0.40824829, -0.40824829]]])

In [8]:

tsteps = 1
batch_size = 1
num_features = 7
model = Sequential()
model.add(LSTM(64,
               input_shape=(1, num_features),
               return_sequences=True,
               stateful=False))
model.add(Dropout(0.33))

model.add(LSTM(64,
               input_shape=(1, num_features),
               return_sequences=False,
               stateful=False))
model.add(Dropout(0.33))

model.add(Dense(4, kernel_initializer='lecun_uniform'))
model.add(Activation('relu')) #linear output so we can have range of real-valued outputs

rms = RMSprop()
adam = Adam()
model.compile(loss='mse', optimizer=adam)

In [9]:
model.predict(init_state(X_train)[0], batch_size=batch_size)

(2400, 1, 7)


array([[ 0.        ,  0.        ,  0.00613082,  0.        ]], dtype=float32)

In [10]:
def act(state, data, action, trades, step):
    if step + 2 == data.shape[0]:
        state = data[step:step+1, 0:1, :]
        term = True
        trades.iloc[timestep+1] = 0
        
        return state, step, trades, term
    state = data[step:step+1, 0:1, :]
    # Take trade action {0:Hold, 1:Buy, 2:Sell}
    if action == 1:
        trades.iloc[step+1] == 1
    elif action == 2:
        trades.iloc[step+1] == -1
    else:
        trades.iloc[step+1 ==0]
    term = False
    step += 1
    return state, step, trades, term

In [11]:
def bestow(new_state, step, action, data, trades, term, epoch = 0):
    bestowal = 0
    trades = trades.fillna(False)
    if term == False:
        b_test = bt.Backtest(pd.Series(data=[i for i in data[step-2:step]], index = trades[step-2:step].index.values),trades[step-2:step], signalType='shares')
        bestowal = ((b_test.data['price'].iloc[-1] -b_test.data['price'].iloc[-2]) * b_test.data['shares'].iloc[-1])
    else:
        b_test = bt.Backtest(pd.Series(data=[i for i in data], index = trades.index.values), trades, signalType='shares')
        reward = b_test.pnl.iloc[-1]
    return bestowal

In [12]:
def eval_Q(eval_data, eval_model, epoch = 0):
    trades = pd.Series(index=np.arange(len(data)))
    state, data, prices = init_state(eval_data)
    step = 1
    term = False
    go = True
    while go:
        q_values = eval_model.predict(state, batch_size = 1)
        action = np.argmax(qval)
        state_prime, step, trades, term = act(state, data, action, trades, step)
        bestowal = bestow(state_prime,step,action,prices, trades, term, epoch = epoch)
        state = state_prime
        if not term:
            go == False
    return bestowal